# Tag to animal frame: re-orienting tag to match animal's axes

Here's a tutorial my friend Max developed to help with this headache: https://flukeandfeather.com/posts/2024-08-30-animal-orientation-with-imu-ta/

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle

# Import necessary pyologger utilities
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
color_mapping_path = os.path.join(root_dir, "color_mappings.json")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.logger_info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

In [ ]:
acc_df = data_pkl.derived_data['calibrated_acc']
mag_df = data_pkl.derived_data['calibrated_mag']
gyr_df = data_pkl.sensor_data['gyroscope']

# Calculate and print sampling frequency for each dataframe
acc_fs = calculate_sampling_frequency(acc_df)
print(f"Accelerometer Sampling frequency: {acc_fs} Hz")

mag_fs = calculate_sampling_frequency(mag_df)
print(f"Magnetometer Sampling frequency: {mag_fs} Hz")

gyr_fs = calculate_sampling_frequency(gyr_df)
print(f"Gyroscope Sampling frequency: {gyr_fs} Hz")

In [ ]:
acc_df

In [ ]:
# acc_data = data_pkl.sensor_data['accelerometer'][['ax','ay','az']]
# mag_data = data_pkl.sensor_data['magnetometer'][['mx', 'my', 'mz']]

acc_data = acc_df[['ax','ay','az']]
mag_data = mag_df[['mx','my','mz']]
gyr_data = gyr_df[['gx', 'gy', 'gz']]

upsampled_columns = []
for col in gyr_data.columns:
    upsampled_col = upsample(gyr_data[col].values, acc_fs / gyr_fs, len(acc_data))  # Apply upsample to each column
    upsampled_columns.append(upsampled_col)  # Append the upsampled column to the list

# Combine the upsampled columns back into a NumPy array
gyr_data_upsampled = np.column_stack(upsampled_columns)
gyr_data = gyr_data_upsampled

acc_data = acc_data.values
mag_data = mag_data.values

# Assuming acc_data and mag_data_upsampled are NumPy arrays
print(f"Gyroscope upsampled to match accelerometer length: acc_data shape= {acc_data.shape}, upsampled gyr_data shape = {gyr_data.shape}")
sampling_rate = acc_fs

In [ ]:
import numpy as np
from magnetic_field_calculator import MagneticFieldCalculator

def orientation_and_heading_correction(abar0, acc_data, mag_data, latitude, longitude, gyr_data=None):
    """
    Corrects the orientation and heading of tag data to align with the reference frame of an animal.

    Parameters
    ----------
    abar0 : array_like
        A 3-element vector representing the accelerometer readings when the animal is stationary on its belly.
    acc_data : array_like
        A 2D array where each row represents a 3-element accelerometer reading.
    mag_data : array_like
        A 2D array where each row represents a 3-element magnetometer reading.
    latitude : float
        Latitude of the location where the tag data was recorded.
    longitude : float
        Longitude of the location where the tag data was recorded.
    gyr_data : array_like, optional
        A 2D array where each row represents a 3-element gyroscope reading. If provided, gyroscope data will also be corrected.

    Returns
    -------
    pitch_deg : array_like
        The pitch angles in degrees for the entire dataset.
    roll_deg : array_like
        The roll angles in degrees for the entire dataset.
    heading_deg : array_like
        The heading angles in degrees for the entire dataset.
    acc_corr : array_like
        The corrected accelerometer data.
    mag_corr : array_like
        The corrected magnetometer data.
    gyr_corr : array_like, optional
        The corrected gyroscope data. Only returned if gyr_data is provided.

    Notes
    -----
    This function seeks to rotate tag data to the reference frame of an animal.
    The function first normalizes the provided stationary accelerometer vector, computes the pitch and roll angles,
    and then applies the corresponding rotation matrices to correct the input accelerometer, magnetometer, and optionally,
    gyroscope data. The function returns the corrected orientation in terms of pitch, roll, and heading angles.

    Examples
    --------
    >>> abar0 = np.array([0.1, 0.2, -0.98])
    >>> acc_data = np.random.rand(100, 3)
    >>> mag_data = np.random.rand(100, 3)
    >>> pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr = orientation_and_heading_correction(abar0, acc_data, mag_data, 32.764567, -117.228665)
    >>> pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr, gyr_corr = orientation_and_heading_correction(abar0, acc_data, mag_data, 32.764567, -117.228665, gyr_data=np.random.rand(100, 3))
    """
    # Normalize abar0 to create abar
    abar = abar0 / np.linalg.norm(abar0)
    
    # Calculate initial pitch (p0) and roll (r0)
    p0 = -np.arcsin(abar[0])
    r0 = np.arctan2(abar[1], abar[2])
    # Constrain p to [-pi / 2, pi / 2]
    if p0 > np.pi / 2:
        p0 = np.pi / 2 - p0
        r0 = r0 + np.pi

    # Define rotation matrices for pitch and roll
    def rotP(p):
        return np.array([[np.cos(p), 0, np.sin(p)],
                         [0, 1, 0],
                         [-np.sin(p), 0, np.cos(p)]])
    
    def rotR(r):
        return np.array([[1, 0, 0],
                         [0, np.cos(r), -np.sin(r)],
                         [0, np.sin(r), np.cos(r)]])
    
    # Calculate rotation matrix W
    W = np.matmul(rotP(p0), rotR(r0)).T

    # Correct the accelerometer and magnetometer data for the entire dataset
    acc_corr = np.matmul(acc_data, W)
    mag_corr = np.matmul(mag_data, W)
    
    # Correct the gyroscope data if provided
    if gyr_data is not None:
        gyr_corr = np.matmul(gyr_data, W)
    else:
        gyr_corr = None
    
    # Calculate magnitude of the corrected accelerometer vectors
    A = np.linalg.norm(acc_corr, axis=1)
    
    # Calculate pitch and roll in degrees from corrected accelerometer data
    pitch_deg = -np.degrees(np.arcsin(acc_corr[:, 0] / A))
    roll_deg = np.degrees(np.arctan2(acc_corr[:, 1], acc_corr[:, 2]))
    
    # Initialize an array to hold the gimbaled magnetic data
    mag_horiz = np.zeros_like(mag_corr)
    
    # Apply the un-roll and un-pitch rotation for each time step
    for i in range(len(pitch_deg)):
        mag_horiz[i, :] = np.matmul(np.matmul(mag_corr[i, :], rotR(np.deg2rad(roll_deg[i])).T), rotP(np.deg2rad(pitch_deg[i])).T)

    # Get the declination using MagneticFieldCalculator
    calculator = MagneticFieldCalculator()
    result = calculator.calculate(latitude=latitude, longitude=longitude)
    declination = result['field-value']['declination']

    print(f"The declination at latitude {latitude} and longitude {longitude} is {declination} degrees.")
    
    # Calculate heading in degrees from corrected magnetometer data
    heading_deg = np.degrees(np.arctan2(mag_horiz[:, 1], mag_horiz[:, 0])) + declination['value']
    
    # Return the corrected pitch, roll, and heading for the entire dataset
    if gyr_corr is not None:
        return pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr, gyr_corr
    else:
        return pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr


abar0 = [0, 0, -9.8]
# Use the function to get corrected orientation and heading for the entire dataset
pitch_deg, roll_deg, heading_deg, corrected_acc, corrected_mag, corrected_gyr = orientation_and_heading_correction(
    abar0, 
    latitude= 32.764655,
    longitude= -117.228585,
    acc_data=acc_data, 
    mag_data=mag_data, 
    gyr_data=gyr_data)


In [8]:
import pandas as pd

# One datetime column from highest sampled data that was matched by other sensors
datetime_data = data_pkl.sensor_data['accelerometer']['datetime']


# Step 1: Create a DataFrame for pitch, roll, and heading
prh_df = pd.DataFrame({
    'datetime': datetime_data,
    'pitch': pitch_deg,
    'roll': roll_deg,
    'heading': heading_deg
})

# Store the 'prh' variable in derived_data
data_pkl.derived_data['prh'] = prh_df
data_pkl.derived_info['prh'] = {
    "channels": ["pitch", "roll", "heading"],
    "metadata": {
        'pitch': {'original_name': 'Pitch (degrees)',
                  'unit': 'degrees',
                  'sensor': 'accelerometer'},
        'roll': {'original_name': 'Roll (degrees)',
                 'unit': 'degrees',
                 'sensor': 'accelerometer'},
        'heading': {'original_name': 'Heading (degrees)',
                    'unit': 'degrees',
                    'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["accelerometer", "magnetometer"],
    "transformation_log": ["calculated_pitch_roll_heading"]
}

# Step 2: Create DataFrames for corrected accelerometer, magnetometer, and gyroscope data
corrected_acc_df = pd.DataFrame({
    'datetime': datetime_data,
    'ax': corrected_acc[:, 0],
    'ay': corrected_acc[:, 1],
    'az': corrected_acc[:, 2]
})

corrected_mag_df = pd.DataFrame({
    'datetime': datetime_data,
    'mx': corrected_mag[:, 0],
    'my': corrected_mag[:, 1],
    'mz': corrected_mag[:, 2]
})

corrected_gyr_df = pd.DataFrame({
    'datetime': datetime_data,
    'gx': corrected_gyr[:, 0],
    'gy': corrected_gyr[:, 1],
    'gz': corrected_gyr[:, 2]
})

# Step 3: Store the corrected accelerometer, magnetometer, and gyroscope data into derived_data
data_pkl.derived_data['corrected_acc'] = corrected_acc_df
data_pkl.derived_info['corrected_acc'] = {
    "channels": ["ax", "ay", "az"],
    "metadata": {
        'ax': {'original_name': 'Acceleration X (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'ay': {'original_name': 'Acceleration Y (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'az': {'original_name': 'Acceleration Z (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'}
    },
    "derived_from_sensors": ["accelerometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_mag'] = corrected_mag_df
data_pkl.derived_info['corrected_mag'] = {
    "channels": ["mx", "my", "mz"],
    "metadata": {
        'mx': {'original_name': 'Magnetometer X (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'my': {'original_name': 'Magnetometer Y (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'mz': {'original_name': 'Magnetometer Z (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["magnetometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_gyr'] = corrected_gyr_df
data_pkl.derived_info['corrected_gyr'] = {
    "channels": ["gx", "gy", "gz"],
    "metadata": {
        'gx': {'original_name': 'Gyroscope X (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gy': {'original_name': 'Gyroscope Y (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gz': {'original_name': 'Gyroscope Z (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'}
    },
    "derived_from_sensors": ["gyroscope"],
    "transformation_log": ["corrected_orientation"]
}


In [ ]:
import pandas as pd

# One datetime column from highest sampled data that was matched by other sensors
datetime_data = data_pkl.sensor_data['accelerometer']['datetime']


# Step 1: Create a DataFrame for pitch, roll, and heading
prh_df = pd.DataFrame({
    'datetime': datetime_data,
    'pitch': pitch_deg,
    'roll': roll_deg,
    'heading': heading_deg
})

# Store the 'prh' variable in derived_data
data_pkl.derived_data['prh'] = prh_df
data_pkl.derived_info['prh'] = {
    "channels": ["pitch", "roll", "heading"],
    "metadata": {
        'pitch': {'original_name': 'Pitch (degrees)',
                  'unit': 'degrees',
                  'sensor': 'accelerometer'},
        'roll': {'original_name': 'Roll (degrees)',
                 'unit': 'degrees',
                 'sensor': 'accelerometer'},
        'heading': {'original_name': 'Heading (degrees)',
                    'unit': 'degrees',
                    'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["accelerometer", "magnetometer"],
    "transformation_log": ["calculated_pitch_roll_heading"]
}

# Step 2: Create DataFrames for corrected accelerometer, magnetometer, and gyroscope data
corrected_acc_df = pd.DataFrame({
    'datetime': datetime_data,
    'ax': corrected_acc[:, 0],
    'ay': corrected_acc[:, 1],
    'az': corrected_acc[:, 2]
})

corrected_mag_df = pd.DataFrame({
    'datetime': datetime_data,
    'mx': corrected_mag[:, 0],
    'my': corrected_mag[:, 1],
    'mz': corrected_mag[:, 2]
})

corrected_gyr_df = pd.DataFrame({
    'datetime': datetime_data,
    'gx': corrected_gyr[:, 0],
    'gy': corrected_gyr[:, 1],
    'gz': corrected_gyr[:, 2]
})

# Step 3: Store the corrected accelerometer, magnetometer, and gyroscope data into derived_data
data_pkl.derived_data['corrected_acc'] = corrected_acc_df
data_pkl.derived_info['corrected_acc'] = {
    "channels": ["ax", "ay", "az"],
    "metadata": {
        'ax': {'original_name': 'Acceleration X (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'ay': {'original_name': 'Acceleration Y (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'az': {'original_name': 'Acceleration Z (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'}
    },
    "derived_from_sensors": ["accelerometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_mag'] = corrected_mag_df
data_pkl.derived_info['corrected_mag'] = {
    "channels": ["mx", "my", "mz"],
    "metadata": {
        'mx': {'original_name': 'Magnetometer X (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'my': {'original_name': 'Magnetometer Y (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'mz': {'original_name': 'Magnetometer Z (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["magnetometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_gyr'] = corrected_gyr_df
data_pkl.derived_info['corrected_gyr'] = {
    "channels": ["gx", "gy", "gz"],
    "metadata": {
        'gx': {'original_name': 'Gyroscope X (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gy': {'original_name': 'Gyroscope Y (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gz': {'original_name': 'Gyroscope Z (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'}
    },
    "derived_from_sensors": ["gyroscope"],
    "transformation_log": ["corrected_orientation"]
}


In [ ]:
OVERLAP_START_TIME = '2024-01-16 09:30:00'  # Start time for plotting
OVERLAP_END_TIME = '2024-01-16 10:30:00'  # End time for plotting
ZOOM_START_TIME = '2024-01-16 10:00:00'  # Start time for zooming
ZOOM_END_TIME = '2024-01-16 10:02:30'  # End time for zooming
TARGET_SAMPLING_RATE = 10

notes_to_plot = {
    'heartbeat_manual_ok': {'sensor': 'ecg', 'symbol': 'triangle-down', 'color': 'blue'},
    'heartbeat_auto_detect_accepted': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'green'},
    'heartbeat_auto_detect_rejected': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'red'}
}

fig = plot_tag_data_interactive5(
    data_pkl=data_pkl,
    sensors=['ecg', 'accelerometer', 'magnetometer'],
    derived_data_signals=['depth', 'corrected_acc', 'corrected_mag', 'prh'],
    channels={},
    time_range=(OVERLAP_START_TIME, OVERLAP_END_TIME),
    note_annotations=notes_to_plot,
    color_mapping_path=color_mapping_path,
    target_sampling_rate=TARGET_SAMPLING_RATE,
    zoom_start_time=ZOOM_START_TIME,
    zoom_end_time=ZOOM_END_TIME,
    zoom_range_selector_channel='depth',
    plot_event_values=[],
)

In [10]:
# Optional: save new pickle file

with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)